In [286]:
import pickle
with open('../data/data2.pickle', 'rb') as f:
  data = pickle.load(f)

In [289]:
texts = data['joinwithout2n1']
my_stop_words = ['']

In [290]:
def flat(l):
    for k in l:
        if not isinstance(k, (list, tuple)):
            yield k
        else:
            yield from flat(k)
texts = list(flat(texts))

In [291]:
print(str(len(texts)) + ' texts')

38650 texts


First step: Deal with password without capital words

In [292]:
from sklearn.feature_extraction.text import TfidfVectorizer

# change the max_df to get different results

tfidf_vectorizer = TfidfVectorizer(max_df=0.9999, max_features=200000,
                                   min_df=0.0001, stop_words=my_stop_words, 
                                   use_idf=True, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(texts)

print(tfidf_matrix.shape)


CPU times: user 253 ms, sys: 5.4 ms, total: 258 ms
Wall time: 257 ms
(38650, 2632)


In [293]:
terms = tfidf_vectorizer.get_feature_names()
terms

['000',
 '0000',
 '001',
 '002',
 '003',
 '005',
 '007',
 '008',
 '009',
 '0101',
 '011',
 '012',
 '0123',
 '0123456789',
 '023',
 '100',
 '1000',
 '101',
 '1010',
 '1012',
 '102',
 '105',
 '107',
 '110',
 '1109',
 '111',
 '1111',
 '111111',
 '112',
 '1122',
 '112233',
 '113',
 '114',
 '117',
 '118',
 '1188',
 '120',
 '1200',
 '1205',
 '1208',
 '121',
 '1212',
 '122',
 '1224',
 '123',
 '123 abc',
 '1231',
 '123123',
 '12321',
 '123321',
 '1234',
 '12344321',
 '12345',
 '123456',
 '1234567',
 '12345678',
 '123456789',
 '1234567890',
 '1235',
 '123654',
 '123654789',
 '123789',
 '124',
 '125',
 '128',
 '129',
 '1302',
 '131',
 '132',
 '133',
 '13579',
 '138',
 '1411',
 '142',
 '144',
 '146',
 '147',
 '147258369',
 '148',
 '1488',
 '150',
 '1501',
 '1502',
 '151',
 '1512',
 '1515',
 '152',
 '155',
 '156',
 '157',
 '158',
 '159',
 '159357',
 '159753',
 '1605',
 '1608',
 '1610',
 '1616',
 '162',
 '164',
 '172',
 '178',
 '1808',
 '181',
 '1810',
 '182',
 '184',
 '189',
 '1902',
 '1905',
 '19

In [295]:
from sklearn.cluster import KMeans
# num of clusters depends on dataset 
num_clusters = 10
km = KMeans(n_clusters=num_clusters)
%time km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

CPU times: user 21.1 s, sys: 22.5 ms, total: 21.1 s
Wall time: 3.53 s


Try to get a matrix

In [296]:
import pandas as pd
df_tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray())
df_tfidf_matrix

,0,1,2,3,4,5,6,7,8,9,...,2622,2623,2624,2625,2626,2627,2628,2629,2630,2631
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [297]:
import pandas as pd
import numpy as np

data = pd.read_csv('../data/38650-username-sktorrent.txt', encoding='ISO-8859-1', header=None)
import re

data.columns = ['original']
    
data['original'] = data['original'].astype(str)

In [298]:
data = data['original'][0:38650]

In [299]:
news = {'username': data,'text': texts,'cluster': clusters}

frame = pd.DataFrame(news, index = [clusters], columns = ['date','text','cluster'])

In [300]:
frame['cluster'].value_counts() #number of films per cluster (clusters from 0 to 4)


0    37255
4      987
8       96
9       89
1       78
6       56
3       28
2       25
7       21
5       15
Name: cluster, dtype: int64

In [302]:
print("Top terms per cluster:")
print()
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    for ind in order_centroids[i, :60]:
        print(' %s' % terms[ind].split(' '), end=',')
    print()
    print()

Top terms per cluster:

Cluster 0 words: ['martin'], ['666'], ['torrent'], ['heslo'], ['1234'], ['jan'], ['david'], ['123456'], ['koko'], ['111'], ['tomas'], ['cek'], ['007'], ['michal'], ['ska'], ['jakub'], ['man'], ['ink'], ['ova'], ['sek'], ['ice'], ['12345'], ['adam'], ['link'], ['123456789'], ['nik'], ['lik'], ['nick'], ['nice'], ['mar'], ['filip'], ['patrik'], ['milan'], ['roman'], ['777'], ['lol'], ['333'], ['and'], ['456'], ['petr'], ['pet'], ['1986'], ['321'], ['daniel'], ['dominik'], ['cik'], ['the'], ['hov'], ['lenka'], ['159'], ['2014'], ['1997'], ['sky'], ['kova'], ['enka'], ['rent'], ['love'], ['jirka'], ['1987'], ['nokia'],

Cluster 1 words: ['peter'], ['123456'], ['123'], ['1610'], ['sur'], ['lia'], ['cross'], ['tru'], ['1972'], ['1111'], ['pan'], ['xxx'], ['tek'], ['denis'], ['1998'], ['1991'], ['1987'], ['1986'], ['123456789'], ['1234'], ['trn'], ['sarin'], ['kovic'], ['val'], ['sky'], ['ova'], ['gag'], ['fry'], ['gap'], ['gargamel'], ['gas'], ['gast'], ['gay'], ['fri

In [180]:
coun = 0
for i in clusters:
    if(i == 0):
        coun = coun+1
coun

974

In [268]:
for i in df_tfidf_matrix[0][1:10]:
    if(i!=0):
        print(i)

In [19]:
news = {'date': dates,'articlecode': articlecodes, 'title': titles,'text': texts,'cluster': clusters}
frame = pd.DataFrame(news, index = [clusters], columns = ['date','articlecode','title','text','cluster'])

Top terms per cluster:

Cluster 0 words:

NameError: name 'vocab_frame' is not defined

https://github.com/llSourcell/Latent_Dirichlet_Allocation/blob/master/NewAnalysis-Normal.ipynb